<a href="https://colab.research.google.com/github/Devikrishna545/Self-Attention-Implementation-ML-/blob/main/self_attention_with_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#self attention implementation with weights
import torch


In [ ]:
inputs = torch.tensor([
    [0.75, 0.42, 0.31],  # dream
    [0.90, 0.50, 0.40],  # big
    [0.20, 0.10, 0.05],  # and
    [0.92, 0.53, 0.41],  # work
    [0.35, 0.20, 0.15],  # for
    [0.05, 0.90, 0.10],  # it (roughly orthogonal/away from others)
])

In [ ]:
labels=["Dream","big","and","work","for","it"]

In [ ]:
#want to generate context vector of word  big
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2 #context vector (hardcoded now)

In [ ]:
#randomly initializing Wq,Wk,Wv
torch.manual_seed(123)
W_query =torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key =torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value =torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [ ]:
W_query

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])

In [ ]:
W_key

Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])

In [ ]:
W_value

Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])

In [ ]:
#for the word Big  in Q,K,V Space
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)
print(key_2)
print(value_2)

tensor([0.4219, 1.1558])
tensor([0.3411, 0.7303])
tensor([0.2737, 0.7088])


In [ ]:
#Calculating Q,K,V USING X,Wq,Wk,Wv

Query = inputs @ W_query
Key = inputs @ W_key
Value = inputs @ W_value

print("Keys shape",Key.shape)
print("Query shape",Query.shape)
print("Value shape",Value.shape)

print(Key)
print(Query)
print(Value)

Keys shape torch.Size([6, 2])
Query shape torch.Size([6, 2])
Value shape torch.Size([6, 2])
tensor([[0.2775, 0.5950],
        [0.3411, 0.7303],
        [0.0615, 0.1275],
        [0.3525, 0.7610],
        [0.1319, 0.2842],
        [0.2040, 0.7276]])
tensor([[0.3507, 0.9452],
        [0.4219, 1.1558],
        [0.0881, 0.2155],
        [0.4361, 1.1954],
        [0.1651, 0.4485],
        [0.2487, 0.7322]])
tensor([[0.2264, 0.5727],
        [0.2737, 0.7088],
        [0.0527, 0.1209],
        [0.2859, 0.7331],
        [0.1075, 0.2733],
        [0.3004, 0.4541]])


In [ ]:
# Key corresponding to the second token and attention of the second token to itself
keys_2 = Key[1]
attention_score22 = query_2.dot(keys_2)
print(attention_score22)

tensor(0.9880)


In [ ]:
#all attention score for query_2(will not add up to one as we didnt normalise)
attention_scores_2 = query_2 @Key.T
print(attention_scores_2)

tensor([0.8047, 0.9880, 0.1733, 1.0283, 0.3842, 0.9271])


In [ ]:
#attention scores without weights
attention_scores = Query @Key.T
print(attention_scores)

tensor([[0.6597, 0.8099, 0.1421, 0.8429, 0.3149, 0.7593],
        [0.8047, 0.9880, 0.1733, 1.0283, 0.3842, 0.9271],
        [0.1527, 0.1874, 0.0329, 0.1950, 0.0729, 0.1748],
        [0.8323, 1.0218, 0.1792, 1.0635, 0.3973, 0.9588],
        [0.3126, 0.3838, 0.0673, 0.3995, 0.1492, 0.3600],
        [0.5046, 0.6195, 0.1086, 0.6449, 0.2409, 0.5835]])


In [ ]:
# scale this by diving by square root of number of dimensions.(for word big)
print(attention_scores_2)
d_k=Key.shape[-1]
attention_weights_2 = torch.softmax(attention_scores_2/d_k**0.5,dim=-1) #(sum of rows sums upto 1)
print(attention_weights_2)
print(d_k)

tensor([0.8047, 0.9880, 0.1733, 1.0283, 0.3842, 0.9271])
tensor([0.1729, 0.1969, 0.1107, 0.2025, 0.1284, 0.1886])
2


In [ ]:
#context vector corresponding to Big
context_vector_2 = attention_weights_2 @ Value
print(context_vector_2)

tensor([0.2272, 0.5212])


In [ ]:
#attenweight for all
attention_weights = torch.softmax(attention_scores/d_k**0.5,dim=-1)
print(attention_weights)

tensor([[0.1724, 0.1917, 0.1196, 0.1962, 0.1351, 0.1850],
        [0.1729, 0.1969, 0.1107, 0.2025, 0.1284, 0.1886],
        [0.1685, 0.1727, 0.1548, 0.1736, 0.1592, 0.1711],
        [0.1730, 0.1978, 0.1090, 0.2037, 0.1272, 0.1892],
        [0.1700, 0.1788, 0.1430, 0.1808, 0.1515, 0.1758],
        [0.1715, 0.1860, 0.1296, 0.1893, 0.1423, 0.1813]])


In [ ]:
#context vector for all
context_vectors = attention_weights @ Value
print(context_vectors)

tensor([[0.2240, 0.5139],
        [0.2272, 0.5212],
        [0.2117, 0.4861],
        [0.2278, 0.5225],
        [0.2158, 0.4952],
        [0.2205, 0.5057]])


#In Python Class

*   Version_1 shows taking random value  as parameters in forward pass
*   Version_2  shows giving bias  with the weights in forward pass (In real     world scenarios we need to have such biases)


In [ ]:
#making everything under python class
import torch.nn as nn

class SelfAttention_version1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_query =nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
        self.W_key =nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
        self.W_value =nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

    def forward(self,x):
      Query = inputs @ self.W_query
      Key = inputs @ self.W_key
      Value = inputs @ self.W_value

      attention_scores = Query @Key.T
      attention_weights = torch.softmax(attention_scores/d_k**0.5,dim=-1)
      context_vectors = attention_weights @ Value
      return context_vectors

In [ ]:
torch.manual_seed(123)
self_attention = SelfAttention_version1(d_in=3,d_out=2)
print(self_attention(inputs))

tensor([[0.2240, 0.5139],
        [0.2272, 0.5212],
        [0.2117, 0.4861],
        [0.2278, 0.5225],
        [0.2158, 0.4952],
        [0.2205, 0.5057]])


In [ ]:
#making everything under python class
import torch.nn as nn

class SelfAttention_version2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super().__init__()
        self.W_query =nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key =nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value =nn.Linear(d_in,d_out,bias=qkv_bias)

    def forward(self,x):
      Query = inputs @ self.W_query.weight.T
      Key = inputs @ self.W_key.weight.T
      Value = inputs @ self.W_value.weight.T

      attention_scores = Query @Key.T
      attention_weights = torch.softmax(attention_scores/d_k**0.5,dim=-1)
      context_vectors = attention_weights @ Value
      return context_vectors

In [ ]:
torch.manual_seed(123)
self_attention = SelfAttention_version2(d_in=3,d_out=2,qkv_bias=True)
print(self_attention(inputs))

tensor([[-0.1048,  0.3039],
        [-0.1065,  0.3039],
        [-0.0995,  0.3038],
        [-0.1066,  0.3040],
        [-0.1012,  0.3038],
        [-0.0934,  0.3073]], grad_fn=<MmBackward0>)
